In [90]:
import pandas as pd
from datetime import datetime
import bcrypt

In [91]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [92]:
df = pd.read_csv('csv/users.csv', encoding='utf-8')

In [101]:
df['num_of_reviews'] = df[['Review1', 'Review2', 'Review3', 'Review4', 'Review5']].notnull().sum(axis=1)

In [102]:
image = pd.read_csv('out/image.csv', encoding='utf-8')

In [103]:
df = df.drop(['21.5.2022', 'Profile picture', 'Review1', 'Review2', 'Review3', 'Review4', 'Review5', 'Review6', 'Shutterstock URL', 'Photo ID'], axis = 1)

In [104]:
df['Username'] = df['Username'].str.replace('\n', '')

In [105]:
df.rename(columns = {'PUBLIC':'hasPublicProfile', 'Nationality' : 'nationality', \
                    'Age' : 'age', 'Sex' : 'sex', 'About Me' : 'bio', 'Name' : 'firstName', \
                    'Surname' : 'lastName', 'Username' : 'username'}, inplace = True)

In [106]:
columns = ['id', 'createdDate', 'updatedDate', 'isActive', 'email', 'hash', 'hasPaswordSaved', \
          'phone', 'statusId', 'defaultLocalityId', 'coverImageId', 'level', 'passwordResetHash', 'passwordResetValid', \
          'searchVector', 'searchConfig', 'defaultCurrencyId', 'imageId']

In [75]:
df = pd.concat([df, pd.DataFrame(columns = columns)])

In [76]:
def set_public_profile(value):
    if value == 'ANO':
        return True
    return False

In [77]:
def create_pass(value):
    temp = 'Trynego+' + str(value).lower()
    bytePwd = temp.encode('utf-8')
    mySalt = bcrypt.gensalt(10)
    hash = bcrypt.hashpw(bytePwd, mySalt)
    return hash

In [111]:
def get_level_id(value):
    temp = int(value) * 2
    if temp < 15:
        return 1
    elif temp < 60:
        return 2
    elif temp < 250:
        return 3
    elif temp < 1000:
        return 4
    elif temp < 3500:
        return 5
    else:
        return 6

In [112]:
df['id'] = [i for i in range(1, df.shape[0]+1)]
df['order'] = ''
df['createdDate'] = dt_string
df['updatedDate'] = dt_string
df['hasPublicProfile'] = df['hasPublicProfile'].apply(set_public_profile)
df['age'] = df.age.astype(int)
df['sex'] = df.sex.str.lower()
df['statusId'] = 4
df['level'] = df['num_of_reviews'].apply(get_level_id)
df['defaultCurrencyId'] = 'USD'

In [79]:
df['hash'] = create_pass('Travelling')

In [80]:
df['isActive'] = True
df['email'] = df.firstName.str.lower() + '.' + df.lastName.str.lower() + '@trynego.com'

In [81]:
def set_image_id(value):
    try:
        return image[:][image['name'] == str(value)]['id'].values[0]
    except:
        return ''

In [82]:
df['imageId'] = df['Photo name'].apply(set_image_id)

In [83]:
df = df.drop(['Photo name'], axis = 1)

In [84]:
df = df.drop_duplicates(subset='username', keep="last")
df = df.drop_duplicates(subset='email', keep="last")

In [85]:
df.shape

(2445, 28)

In [86]:
df.at[47,'id'] = 2449

In [88]:
df.at[df.loc[df['lastName'] == 'Rochette'].index[0], 'username'] = 'Monique Rochette'
df.at[df.loc[df['lastName'] == 'Rochette'].index[0], 'new_name'] = '0868MoniqueRochette'

In [89]:
df.to_csv('out/users.csv', index=False, encoding='utf-8')